In [75]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import clear_output, display, HTML

mode_widg = widgets.RadioButtons(
    options=['text.proc', 'facet.proc'],
                               )

results_cap = widgets.RadioButtons(
    options=['100', '500', 'all'],
                                )

facet_widg = widgets.RadioButtons(
    options=['provider.name', 'sourceResource.format', 'sourceResource.spatial.name', 'sourceResource.subject.name'],
                               )

text_widg = widgets.Text(value="Search")

container = widgets.ContainerWidget(children=[text_widg, mode_widg, facet_widg, results_cap])

display(container)


In [143]:
import json, requests

with open ("dpla.key.txt", "r") as myfile:
    key=myfile.read().replace('\n','')
    
url = ("http://api.dp.la/v2/items?q=" + text_widg.value + "&api_key=" + key + 
        "&facets=" + facet_widg.value + "&page_size=" + results_cap.value)

resp = requests.get(url=url)
results = json.loads(resp.text)

In [87]:
results['count']

276

In [49]:
url

'http://api.dp.la/v2/items?q=kittens&api_key=58ecb0a79f4a704d43a9594d0bb435fb&facets=sourceResource.subject.name&page_size=500'

In [94]:
results['facets']

{'sourceResource.spatial.name': {'_type': 'terms',
  'missing': 169,
  'other': 36,
  'terms': [{'count': 10, 'term': 'USA'},
   {'count': 8, 'term': 'United States'},
   {'count': 8, 'term': 'New York'},
   {'count': 8, 'term': 'Los Angeles'},
   {'count': 6, 'term': 'Charleston (S.C.)'},
   {'count': 6, 'term': 'Brooklyn, N.Y.'},
   {'count': 4, 'term': 'Wisconsin'},
   {'count': 4, 'term': 'New York (State), Western'},
   {'count': 4, 'term': 'Erie County (N.Y.)'},
   {'count': 4, 'term': 'Berkshire County'},
   {'count': 4, 'term': 'Atlanta (Ga.)'},
   {'count': 3, 'term': 'United States--New York (State)--New York (N.Y.)'},
   {'count': 3, 'term': 'United States--New York (State)--Buffalo (N.Y.)'},
   {'count': 3, 'term': 'United States--New Hampshire--Concord (N.H.)'},
   {'count': 2, 'term': 'Virginia'},
   {'count': 2, 'term': 'United States - Texas - Taylor County - Abilene'},
   {'count': 2,
    'term': 'United States - Tennessee - Claiborne County - Cumberland Gap'},
   {'co

In [97]:
import pandas as pd

facets = pd.DataFrame(results['facets'][facet_widg.value]["terms"])

In [98]:
facets

,count,term
0,10,USA
1,8,United States
2,8,New York
3,8,Los Angeles
4,6,Charleston (S.C.)
5,6,"Brooklyn, N.Y."
6,4,Wisconsin
7,4,"New York (State), Western"
8,4,Erie County (N.Y.)
9,4,Berkshire County


In [99]:
import plotly.plotly as py
from plotly.graph_objs import *

facets = facets.sort(['count'])

data = Data([
    Bar(
        x=facets['term'],
        y=facets['count']
    )
])
#plot_url = py.plot(data, filename='basic-bar')
py.iplot(data, filename = 'basic-bar')

In [100]:
%run text.extract.py

In [103]:
raw = ""
records = []
for doc in results["docs"]:
    text = item_parser(doc)
    raw = raw + text + "\n"
    records.append(text)
    #print(text)
    

In [102]:
len(records)

276

In [107]:
import nltk
import pickle
import re
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk.corpus import stopwords
from nltk import bigrams

words = re.split('\W+', raw)
filtered = [w.lower() for w in words if w.lower() not in nltk.corpus.stopwords.words('english')]
fd = nltk.FreqDist((token) for token in filtered)
bgs = list(bigrams(filtered))
bfd = nltk.FreqDist(bgs)
cfd = nltk.ConditionalFreqDist(bgs)

In [108]:
fd.most_common(10)

[('jl', 617),
 ('bm', 542),
 ('kittens', 353),
 ('yeah', 247),
 ('know', 186),
 ('little', 161),
 ('oh', 158),
 ('go', 141),
 ('would', 128),
 ('well', 127)]

In [109]:
bfd.most_common(10)

[(('jl', 'yeah'), 133),
 (('janet', 'quinney'), 84),
 (('quinney', 'lawson'), 84),
 (('oral', 'history'), 76),
 (('land', 'use'), 73),
 (('use', 'management'), 73),
 (('um', 'hmm'), 72),
 (('management', 'oral'), 71),
 (('project', 'janet'), 71),
 (('jl', 'oh'), 71)]

In [110]:
cfd[fd.most_common(1)[0][0]].most_common(10)

[('yeah', 133),
 ('oh', 71),
 ('well', 34),
 ('uh', 23),
 ('bm', 22),
 ('yes', 20),
 ('um', 14),
 ('know', 14),
 ('think', 10),
 ('right', 8)]

In [118]:
print(records[1])

circa 1873 Photographer located in Boston. Animals. Pets. Cats. Kittens 


In [120]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk.corpus import stopwords
# remove common words and tokenize
#stoplist = set('for a of the and to in'.split())
stoplist = nltk.corpus.stopwords.words('english')
texts = [[word for word in record.lower().split() if word not in stoplist]
        for record in records]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
from pprint import pprint   # pretty-printer
#pprint(texts[1:10])

In [147]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                               num_topics=5, update_every=1, 
                               chunksize=10000, passes=10)

In [148]:
lda.print_topics(5)

['0.026*/ + 0.023*kittens + 0.013*fiction + 0.013*cats + 0.010*little + 0.009*children + 0.008*negative + 0.008*animals + 0.007*book + 0.007*three',
 '0.008*kittens + 0.008*new + 0.007*therapy + 0.006*service + 0.006*library + 0.005*dog + 0.004*advertising + 0.004*card + 0.004*angeles + 0.004*dr.',
 '0.025*kittens + 0.010*two + 0.009*(n.c.) + 0.009*cougar + 0.008*new + 0.007*little + 0.007*- + 0.007*animals + 0.007*card + 0.006*county',
 '0.041*jl: + 0.036*bm: + 0.020*– + 0.009*don’t + 0.008*it’s + 0.008*go + 0.007*that’s + 0.007*would + 0.007*yeah. + 0.006*yeah,',
 '0.013*- + 0.011*kittens + 0.010*golden + 0.009*little + 0.008*record + 0.007*records + 0.007*book + 0.006*united + 0.006*states + 0.006*cat']

In [146]:
lda.print_topics(5)

['0.010*- + 0.008*two + 0.008*one + 0.008*kittens + 0.006*children + 0.005*kittens, + 0.005*united + 0.005*san + 0.004*film + 0.004*air',
 '0.013*little + 0.012*kittens + 0.008*book + 0.008*golden + 0.007*new + 0.007*therapy + 0.006*record + 0.006*records + 0.005*cats + 0.004*negative',
 '0.008*library + 0.008*service + 0.008*new + 0.007*fiction + 0.004*two + 0.004*kittens + 0.004*name + 0.004*special + 0.004*family + 0.003*also',
 '0.041*jl: + 0.035*bm: + 0.020*– + 0.009*don’t + 0.008*it’s + 0.008*go + 0.007*that’s + 0.007*would + 0.007*yeah. + 0.006*yeah,',
 '0.034*kittens + 0.015*/ + 0.012*- + 0.010*card + 0.010*cats + 0.008*animals + 0.008*(n.c.) + 0.007*cat + 0.007*cougar + 0.007*x']

In [149]:
for doc_bow in corpus[1:10]:
    print(lda[doc_bow])

[(0, 0.63263668038880927), (1, 0.025091002602462543), (2, 0.0253986509064749), (3, 0.025002193788302537), (4, 0.29187147231395089)]
[(0, 0.95775112214025149), (1, 0.010581484643899535), (2, 0.010581058988939187), (3, 0.010529206463736911), (4, 0.01055712776317274)]
[(2, 0.97029852825469309)]
[(2, 0.9691568641364211)]
[(2, 0.97135542152850796)]
[(0, 0.86623838001698983), (1, 0.033466248416849559), (2, 0.033538440562674403), (3, 0.03333618237118547), (4, 0.033420748632300809)]
[(2, 0.97030092833937054)]
[(0, 0.8662384529998377), (1, 0.03346624777984715), (2, 0.033538369051601354), (3, 0.033336182363734784), (4, 0.033420747804979105)]
[(0, 0.96176586887642856)]


In [150]:
for text in texts[1:10]:
    print(text)
    print(lda[dictionary.doc2bow(text)])

['circa', 'photographer', 'located', 'animals.', 'pets.', 'cats.', 'kittens']
[(0, 0.63261590210971663), (1, 0.025091000100592876), (2, 0.025383441903617819), (3, 0.025002193093936521), (4, 0.2919074627921362)]
['title', 'information', 'provided', 'leslie', 'jones', 'boston', 'public', 'library', 'negative', 'negative', 'sleeve.', 'date', 'supplied', 'cataloger.', 'cats', 'baby', 'animals', 'kittens']
[(0, 0.95775234692541444), (1, 0.0105810036696123), (2, 0.010580351893497736), (3, 0.010529206321135728), (4, 0.010557091190339696)]
['1970s-1980s', 'cougar', 'cougar', 'kittens', 'grandfather', 'mountain,', 'nc', 'wildlife', 'habitat.', '1970-1979', '1980-1989', 'pumas', 'baby', 'animals', 'zoo', 'animals', 'grandfather', 'mountain', '(n.c.)', 'linville', '(n.c.)', 'avery', 'county', '(n.c.)', 'cougar', 'kittens']
[(2, 0.97030098291477651)]
['1970s-1980s', 'cougar', 'cougar', 'kittens', 'enclosure', 'grandfather', 'mountain,', 'nc.', '1970-1979', '1980-1989', 'pumas', 'zoo', 'animals', '

In [155]:
print(lda[dictionary.doc2bow(['little', 'golden', 'book', 'about', 'kittens'])])

[(0, 0.04136557301673438), (1, 0.040314888447378114), (2, 0.040969189222580631), (3, 0.040130460320295692), (4, 0.83721988899301125)]
